In [1]:
from    astropy.table       import Table
import  numpy               as     np
from    astropy.io          import fits
import  matplotlib.pyplot   as     plt
from    astropy.coordinates import Angle        
from    astropy             import units as u             
import  matplotlib.colors   as     colors               

from    tqdm                import tqdm
import  os
import  gc                                         


#this deals with warnings
import warnings                                   
from astropy.io.fits.verify import VerifyWarning                          
warnings.simplefilter('ignore', VerifyWarning)
warnings.simplefilter('ignore', UserWarning) 

#this is just a handy little function to return the desired file path
#give one entry in the objectlist, return the desired file path
def file_path(obj,prefix,filetype='fits'):
    if   filetype == 'fits':
        return f"data\\{obj['field']}\\{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"
    elif filetype == 'png':
        return f"png\\{obj['field']}\\{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"

### define single plot function

In [ ]:
def single_plot(disp,ra,dec,pix,title,ax,style='gray'):

    norm = colors.Normalize(vmin=0)
    im    =  ax.imshow(disp
                    ,norm = norm
                    ,origin='lower', cmap = style)
    fig.colorbar(im,ax=ax)
    
    
    plt.title(f'{title}, pixel scale: {pix}\'')
    # Set axis labels
    ax.set_xlabel(f"Ra  +  {Angle(round(ra,4),unit='deg').to_string(unit=u.degree, sep=('h', 'm', 's'))}")
    ax.set_ylabel(f"Dec + {Angle(round(dec,4),unit='deg').to_string(unit=u.degree, sep=('deg', 'm', 's'))}")
    
    # Add N and E directions with arrows
    ax.text(0.9, 0.9, 'N', transform=ax.transAxes, ha='center', va='center', color='white', fontsize=14)
    ax.text(0.1, 0.1, 'E', transform=ax.transAxes, ha='center', va='center', color='white', fontsize=14)

In [1]:
obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')
step = 0

for index in tqdm(range(len(obj_lis[:10])),
                    desc="Processing Table",
                    ncols=100):
    obj=obj_lis[index]


    if os.path.exists(file_path(obj=obj,prefix='fullplot',filetype='png')):
        continue
    
    path = file_path(obj=obj,prefix='extracted')
    with fits.open(path) as extracted:
        #mask = extracted[2].data 
        #mask[mask==extracted[0].header['id']] = 1
        #mask[mask!=1] = 0

# this the part for plot the calculated result
        ra    =  obj['ra']
        dec   =  obj['dec']
        pix   =  extracted[3].header['PIXASEC'] #arcsec

        fig = plt.figure(figsize=(15,15))   
        stuff_to_plot = [3,10,4,6,11,12]
        for index in range(len(stuff_to_plot)):
            #parameter control
            i = stuff_to_plot[index]    
            disp  =  extracted[i].data
            title =  obj['field'],obj['id'],extracted[i].name
            ax    =  fig.add_subplot(int(f'32{index+1}')) 
            single_plot(disp,ra,dec,pix,title,ax,style='binary_r')
        plt.savefig(file_path(obj=obj,prefix='fullplot',filetype='png'))
        plt.close('all')
        step += 1
        if step >= 10:
            gc.collect()   
            step=0  


NameError: name 'Table' is not defined